Conexão do Psycopg com o Banco de dados

In [2]:
import psycopg2

def conexao():
    try:
        
        conn = psycopg2.connect(
            host="200.129.44.249", 
            database="553095", 
            user="553095", 
            password="553095"
        )
        return conn
    except psycopg2.Error as e:
        print("Ocorreu um erro ao conectar: ", e)
        return None
    


Criação Tabelas 2.2

In [16]:

conn = conexao()
if conn is None:
    print("Conexão não estabelecida")
    exit(1)
else: 
    cur = conn.cursor()

    try:
        cur.execute("DROP TABLE IF EXISTS public.Curso CASCADE")
        cur.execute("CREATE TABLE public.Curso(id INT, nome VARCHAR(100), regime VARCHAR(100), duracao INT, primary key(id))")
        conn.commit()   

        cur.execute("DROP TABLE IF EXISTS public.Aluno CASCADE")
        cur.execute("CREATE TABLE public.aluno(id INT, nome VARCHAR(100), curso_id INT, semestre INT, PRIMARY KEY(id), FOREIGN KEY(curso_id) REFERENCES Curso(id))")
        conn.commit()   

        cur.execute("DROP TABLE IF EXISTS public.Professor CASCADE")
        cur.execute("CREATE TABLE public.Professor(id INT, nome VARCHAR(100), area_especializacao VARCHAR(100), contato VARCHAR(100), curso_id INT, PRIMARY KEY(id), FOREIGN KEY(curso_id) REFERENCES Curso(id))")
        conn.commit()   

        cur.execute("DROP TABLE IF EXISTS public.Disciplina CASCADE")
        cur.execute("CREATE TABLE public.Disciplina(id INT, codigo VARCHAR(10) UNIQUE, nome VARCHAR(100), area_especializacao VARCHAR(100), Carga_horaria INT, curso_id INT, PRIMARY KEY(id), FOREIGN KEY(curso_id) REFERENCES Curso(id))")
        conn.commit()   

        cur.execute("DROP TABLE IF EXISTS public.Turma CASCADE")
        cur.execute("CREATE TABLE public.Turma(id INT, codigo VARCHAR(10) UNIQUE, disciplina_id INT, semestre VARCHAR(20), capacidade_maxima INT, estado VARCHAR(20), prof_id INT, PRIMARY KEY(id), FOREIGN KEY(disciplina_id) REFERENCES Disciplina(id), FOREIGN KEY(prof_id) REFERENCES Professor(id))")
        conn.commit()   

        cur.execute("DROP TABLE IF EXISTS public.Aluno_Turma CASCADE")
        cur.execute("CREATE TABLE public.Aluno_Turma(aluno_id INT, turma_id INT, PRIMARY KEY(aluno_id, turma_id), FOREIGN KEY(aluno_id) REFERENCES Aluno(id), FOREIGN KEY(turma_id) REFERENCES Turma(id))")
        conn.commit()

        cur.close()
        print("Tabelas criadas com sucesso")

    except Exception as e:
        print("Erro: ", e)
        conn.rollback()
    finally:
        if conn is not None:
            conn.close()
            print("Conexão encerrada")    
    
        




Tabelas criadas com sucesso
Conexão encerrada


Inserção de Dados 2.3


In [17]:

conn = conexao()
if conn is None:
    print("Conexão não estabelecida")
    exit(1)
else: 
    cur = conn.cursor()

    try:
        cur.execute("Insert into public.Curso(id, nome, regime, duracao) values(1, 'Ciências da Computação', 'Semestral', 8), (2, 'Engenharia de Software', 'Anual', 10), (3, 'Sistemas de Informação', 'Semestral', 8) ")
        conn.commit()   

        cur.execute("Insert into public.Aluno(id, nome, curso_id, semestre) values(1, 'João Silva', 1, 1), (2, 'Maria Costa', 1, 1), (3, 'Ana Souza', 3, 5), (4, 'Pedro Almeida', 2, 3), (5, 'Lucas Santos', 2, 3)")
        conn.commit()   

        cur.execute("Insert into public.Professor(id, nome, area_especializacao, contato, curso_id) values(1, 'Maria Oliveira', 'Banco de Dados', 'maria@ufc.br', 1), (2, 'João Pereira', 'Redes de Computadores', 'joao@ufc.br', 2), (3, 'Ana Silva', 'Inteligência Artificial', 'ana@ufc.br', 3), (4, 'Paulo Santos', 'Engenharia de Software', 'paulo@ufc.br', 2), (5, 'Carla Mendes', 'Redes de Computadores', 'carla@ufc.br', 1)")
        conn.commit()   

        cur.execute("Insert into public.Disciplina(id, codigo, nome, area_especializacao, carga_horaria, curso_id) values(1, 'BD001', 'Fundamentos de Bancos de Dados', 'Banco de Dados', 60, 1), (2, 'IA002', 'Inteligência Computacional Aplicada', 'Inteligência Artificial', 80, 3), (3, 'RS003', 'Segurança da Informação', 'Redes de Computadores', 40, 2), (4, 'BD004', 'Introdução à Ciência de Dados', 'Banco de Dados', 60, 1), (5, 'ES005', 'Qualidade de Software', 'Engenharia de Software', 50, 2)")
        conn.commit()   

        cur.execute("Insert into public.Turma(id, codigo, disciplina_id, semestre, capacidade_maxima, estado, prof_id) values(1, 'CC2024BD1', 1, '2024.2', 4, 'Aberta', 1), (2, 'CC2024IA1', 2, '2024.2', 4, 'Aberta', 3), (3, 'CC2024RS1', 3, '2024.1', 8, 'Aberta', 2), (4, 'CC2024DS1', 4, '2024.2', 4, 'Aberta', 1), (5, 'CC2024ES1', 5, '2024.2', 8, 'Aberta', 4)")
        conn.commit()   

        cur.execute("Insert into public.Aluno_Turma(aluno_id, turma_id) values(1, 1), (2, 1), (3, 2), (4, 3), (5, 4), (1, 5), (2, 4), (3, 5), (4, 2), (5, 3)")
        conn.commit()

        cur.close()
        print("Registros inseridos com sucesso")
        
    except Exception as e:
        print("Erro: ", e)
        conn.rollback()
    finally:
        if conn is not None:
            conn.close()
            print("Conexão encerrada")


Registros inseridos com sucesso
Conexão encerrada


Consulta 2.4

In [7]:
querry = []

conn = conexao()
if conn is None:
    print("Conexão não estabelecida")
    exit(1)
else: 
    cur = conn.cursor()

    try:
        cur.execute("SELECT t.*, COUNT(t.id) as total_alunos FROM public.turma t, public.aluno_turma alt WHERE t.id = alt.turma_id GROUP BY(t.id)")
        querry.append(cur.fetchall())

        cur.execute("SELECT a.*, t.disciplina_id, d.nome FROM public.turma t, public.aluno_turma alt, public.aluno a, public.disciplina d WHERE a.id = alt.aluno_id and t.id = alt.turma_id and t.disciplina_id = d.id and d.nome = 'Fundamentos de Bancos de Dados'")
        querry.append(cur.fetchall())

        cur.execute("SELECT COUNT (c.id) FROM public.professor p, public.curso c WHERE p.curso_id = c.id and c.nome = 'Ciências da Computação'")
        querry.append(cur.fetchall())

        for i in querry:
            print(i)

        conn.commit()
        cur.close()
    except Exception as e:
        print("Erro: ", e)
        conn.rollback()
    finally:
        if conn is not None:
            conn.close()
            print("Conexão encerrada")


[(5, 'CC2024ES1', 5, '2024.2', 8, 'Aberta', 4, 2), (4, 'CC2024DS1', 4, '2024.2', 4, 'Aberta', 1, 2), (2, 'CC2024IA1', 2, '2024.2', 4, 'Aberta', 3, 2), (1, 'CC2024BD1', 1, '2024.2', 4, 'Aberta', 1, 2), (3, 'CC2024RS1', 3, '2024.1', 8, 'Aberta', 2, 2)]
[(1, 'João Silva', 1, 1, 1, 'Fundamentos de Bancos de Dados'), (2, 'Maria Costa', 1, 1, 1, 'Fundamentos de Bancos de Dados')]
[(2,)]
Conexão encerrada


Transação 2.5

In [26]:

conn = conexao()
if conn is None:
    print("Conexão não estabelecida")
    exit(1)
else: 
    cur = conn.cursor()

    try:
        cur.execute("UPDATE public.turma SET estado = 'Fechada' WHERE codigo = 'CC2024DS1'")
        cur.execute("DELETE FROM public.aluno_turma WHERE turma_id = 4")

        conn.commit()
        cur.close()
        print("Transação realizada com sucesso")
    except Exception as e:
        print("Erro: ", e)
        conn.rollback()
    finally:
        if conn is not None:
            conn.close()
            print("Conexão encerrada")



Transação realizada com sucesso
Conexão encerrada


Procedimento Armazenado 2.6

In [8]:
conn = conexao()
if conn is None:
    print("Conexão não estabelecida")
    exit(1)
else: 
    cur = conn.cursor()

    try:
        cur.execute("CALL inc_semestre(1)")
        conn.commit()
        cur.close()
        print("Procedimento realizado com sucesso")
    except psycopg2.Error as e:
        print("Erro: ", e)
        conn.rollback()
    finally:
        if conn is not None:
            conn.close()
            print("Conexão encerrada")
   

Procedimento realizado com sucesso
Conexão encerrada


Trigger 2.7.3

In [18]:

conn = conexao()
if conn is None:
    print("Conexão não estabelecida")
    exit(1)
else: 
    cur = conn.cursor()
    
    try:
        cur.execute("CREATE TRIGGER trig_verificar_capacidade_turma BEFORE INSERT ON aluno_turma FOR EACH ROW EXECUTE FUNCTION verificar_capacidade_turma();")
        conn.commit()
    except psycopg2.Error as e:
        print("Erro: ", e)
        conn.rollback()

    try:
        cur.execute("Insert into public.Aluno_Turma(aluno_id, turma_id) values(3, 1), (5, 1), (4, 1)")
        conn.commit()
        cur.close()
    except psycopg2.Error as e:
        print("Erro: ", e)
        conn.rollback()
    finally:
        if conn is not None:
            conn.close()
            print("Conexão encerrada")
 

Erro:  A turma 1 já atingiu a capacidade máxima.
CONTEXT:  PL/pgSQL function verificar_capacidade_turma() line 14 at RAISE

Conexão encerrada


Trigger 2.7.4

In [19]:
conn = conexao()
if conn is None:
    print("Conexão não estabelecida")
    exit(1)
else:
    cur = conn.cursor()

    try:
        cur.execute("CREATE TRIGGER trig_verificar_maximo_disciplina BEFORE INSERT ON aluno_turma FOR EACH ROW EXECUTE FUNCTION verificar_maximo_disciplina();")
        conn.commit()
        
    except psycopg2.Error as e:
        print("Erro: ", e)
        conn.rollback()

    try:
        cur.execute("Insert into public.Aluno_Turma(aluno_id, turma_id) values(1, 2), (1, 3), (1, 4)")
        conn.commit()
        cur.close()
    except psycopg2.Error as e:
        print("Erro: ", e)
        conn.rollback()
    finally:
        if conn is not None:
            conn.close()
            print("Conexão encerrada")
    

Erro:  O aluno 1 já está matriculado em 4 disciplinas nesse semestre.
CONTEXT:  PL/pgSQL function verificar_maximo_disciplina() line 8 at RAISE

Conexão encerrada
